**Sustainable Software Development, block course, March 2021**  
*Scientific Software Center, Institute for Scientific Computing, Dr. Inga Ulusoy*

# Analysis of the data

Imagine you perform a "measurement" of some type and obtain "scientific data". You know what your data represents, but you have only a vague idea how different features in the data are connected, and what information you can extract from the data.

You would start first with going through the data, making sure your data set is complete and that the result is reasonable. Imagine this already happened.

In the next step, you would inspect your data more closely and try to identify structures. That is the step that we are focusing on in this unit.

In the `data` folder, you will find several data files (`*.t` and `*.dat`). These are data files generated through some "new approach" that hasn't been used in your lab before. No previous analysis software exists, and you are going to establish a protocol for this "new approach" and "publish your results".

The data can be grouped into two categories: 
1. data to be analyzed using statistical methods;
2. data to be analyzed using numerical methods.

In your hypothetical lab, you are an "expert" in one particular "method", and your co-worker is an "expert" in the other. Combined these two methods will lead to much more impactful results than if only one of you analyzed the data. Now, the task in this course is to be solved collaboratively with your team member working on one of the analysis approaches, and you working on the other. You will both implement functionality into the same piece of "software", but do so collaboratively through git.

As you do not know yet which analysis is most meaningful for your data, and how to implement it, you will start with a jupyter notebook. You and your team member will work on the same notebook that will be part of a github repository for your project. This is the task for today. Discuss with your team members who will work on the statistical and who on the numerical analysis.

## Step 1

Generate a github repository with the relevant files.

## Step 2

Clone the repository to your local machine.

## Step 3

Start working on task 1 for your analysis approach. 

## Step 4

Create your own branch of the repository and commit your changes to your branch; push to the remote repository.

## Step 5

Open a `pull request` so your team member can review your implementation. Likewise, your team member will ask you to review theirs.

## Step 6

Merge the changes in your branch into `main`. Resolve conflicts.

## Step 7

Repeat working on task; committing and pushing to your previously generated branch or a new branch; open a pull request; merge with main; until you have finished all the tasks in your analysis approach. Delete obsolete branches.

# Start of the analysis notebook

**Author : Your Name**  
*Date : The date you started working on this*  
*Affiliation : The entity under whose name you are working on this*  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
import matplotlib.pyplot as plt
from numpy import *
import pandas as pd
import seaborn as sns

In [ ]:
# a tuple containing the file names
filenames = 'npop.t', 'efield.t', 'expec.t', 'table.dat', 'nstate_i.t'
filedir = '../../data/'
# maximum number of plots generated with seaborn (only first thresh-1 columns are plotted at most)
thresh = 50
# threshold for variance so that feature is evaluated as significant
threshd = 1.0E-5
# threshold for value to be considered different than zero
threshv = 1.0E-5

# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

In [ ]:
# function to read in the data files
def read_in_df(filename, filedir):
    name = '{}{}'.format(filedir,filename)
    print('Reading from file {}'.format(name))
    temp = pd.read_csv(name, '\s+')
    return temp 

In [ ]:
# function to read in the data files
def read_in(filename, filedir):
    name = '{}{}'.format(filedir,filename)
    print('Reading from file {}'.format(name))
    vals = loadtxt(name, skiprows=1)
    vals = vals.T
    return vals 

### Task 1: Read in expect.t and plot relevant data

In [ ]:
df1 = read_in_df(filenames[2],filedir)

In [ ]:
df1.head(10)

In [ ]:
# plot the dataframe using seaborn
sns.relplot(data = df1, kind="line", x="time", y="<z>")
plt.show()

In [ ]:
sns.pairplot(df1, corner=True)
plt.show()

We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
df1 = df1.drop(["norm","<x>","<y>"], axis = 1)

### Task 2: Create plots of the relevant data and save as .pdf.

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
df2 = read_in_df(filenames[1],filedir)
df2.head(10)

In [ ]:
# discard all columns with variance below a set threshold - we can consider them as constant
df2.var()

In [ ]:
df2 = df2.drop(df2.var()[df2.var() < threshd].index.values, axis=1)

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

In [ ]:
sns.lineplot(x='time', y='value', hue='variable', 
             data=pd.melt(df2, ['time']))
plt.show()

In [ ]:
sns.pairplot(df2, kind='kde', corner=True)
plt.show()

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
print("Correlation Matrix")
print(df2.corr())

This tells us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
def get_correlation_measure(df):
    drop_values = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1): # get rid of all diagonal entries and the lower triangular
            drop_values.add((cols[i], cols[j]))
    return drop_values
df2_short = df2.drop(["time"], axis = 1) # get rid of time column
drop_vals = get_correlation_measure(df2_short) # get rid of time column
corr2 = df2_short.corr().unstack()
corr2 = corr2.drop(labels=drop_vals).sort_values(ascending=False, key=lambda col: col.abs()) # sort by absolute values but keep sign
print(corr2[0:12])                                                

Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Generate graphs of the relevant data and save as .pdf.

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`

In [ ]:
values = read_in(filenames[3],filedir)

In [ ]:
# no need for the first two columns, and replace the NaNs by zero
table_vals = delete(values,[0,1],axis=0)
table_vals = nan_to_num(table_vals)

Now calculate how different the vectors in column 1 are from column 0, column 3 from column 2, and column 5 from column 4.

In [ ]:
def calc_euc_dist(list_ref,list_comp,array_in):
    distances = zeros(len(list_ref))
    for i in range(len(list_ref)):
        distances[i] = linalg.norm(array_in[list_comp[i]]-array_in[list_ref[i]])
    return distances

In [ ]:
out_dist = calc_euc_dist([0,2,4],[1,3,5],table_vals)
print(out_dist)
x = range(0,len(out_dist))
plt.bar(x,out_dist)
plt.xticks(x, ('x', 'y', 'z'))
plt.show()

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

In [ ]:
# function to plot the data
def plot_vals(values,thresh):
    if len(values) <= thresh:
        maxcol = len(values)
    else:
        maxcol = thresh
    for i in range(1,maxcol):
        plt.plot(values[0],values[i])
        plt.title('column {}'.format(i))
        plt.show()

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
# read in first file
values = read_in(filenames[1],filedir)

In [ ]:
# plot first set of values
plot_vals(values,thresh)

Here we are interested in column 2 since the others are constant.

In [ ]:
# do the FT - see https://numpy.org/doc/stable/reference/routines.fft.html
def do_fft(data,tmax):
    data_s = fft.rfft(data)
    data_w = fft.rfftfreq(tmax)
    # only take the positive frequency components through rfft
    return data_w, data_s 

In [ ]:
# determine which columns are important
myvar = var(values, axis=1)
print(myvar)
# delete those below threshold
type(myvar)
values = values[nonzero(myvar > threshd)]
print(values)


In [ ]:
om, sig = do_fft(values[1],len(values[0]))
# at this point not sure why the shift is required
# I remember having issues with that previously
# could be the time step is too large
delta = (om[1] - om[0])/2
om = om + delta
plt.plot(om, real(sig), label = 'real part')
plt.plot(om, imag(sig), label = 'imaginary part')
plt.plot(om, abs(sig)**2, label = 'power spectrum')
plt.axvline(0.116, 0, 1, color = 'black', label = "carrier frequency")
plt.xlabel('energy (Eh)')
plt.ylabel('intensity (arb. u.)')
plt.legend()
plt.show()

### Task 2: Generate a plot of your results to be saved as pdf.

### Task 3: Calculate the autocorrelation function from nstate_i.t

In [ ]:
wavef = read_in(filenames[4],filedir)

In [ ]:
# store the time column in a vector and drop from array
time = wavef[0]
wavef = delete(wavef,[0],axis=0)
print(wavef[0])

In [ ]:
# convert to complex array
realpart = wavef[0::2]
imagpart = wavef[1::2]
wavefc = realpart + 1j*imagpart
print(wavefc[0])
print(realpart[0])
print(imagpart[0])

In [ ]:
# Now construct overlap between first vector and all others
def calc_auto(wavef):
    aucofu = zeros(len(wavef[0]),dtype = complex)
    for i in range(0,len(wavef[0])):
        aucofu[i] = sum(wavef[:,0]*conjugate(wavef[:,i]))
    return aucofu

In [ ]:
aucofu = calc_auto(wavefc)
print(aucofu)
plt.plot(abs(aucofu**2))

### Task 4: Generate a plot of your results to be saved as pdf.

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# now do the FT
# do the FT - see https://numpy.org/doc/stable/reference/routines.fft.html
def do_fft(data,tmax):
    data_s = fft.fft(data)
    data_w = fft.fftfreq(tmax)
    # only take the positive frequency components
    return data_w[0:tmax//2], data_s[0:tmax//2] 

energy, spec = do_fft(aucofu,len(time))
print(energy)

In [ ]:
#plt.plot(energy,real(spec))
#plt.plot(energy,imag(spec))
#plt.plot(energy,abs(spec))
plt.plot(energy,abs(spec)**2)
plt.ylim(-0.1,15)

### Task 6: Generate a plot of your results to be saved as pdf.